# Aula 11 - Explicações em Sistemas de Recomendação - Exercícios

## Importação dos dados (MovieLens 100k)

In [1]:
import pandas as pd
import numpy as np

In [6]:
import wget
!python3 -m wget https://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip ml-100k.zip


100% [......................................................] 4924029 / 4924029
Saved under ml-100k.zip
Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflating: ml-100k/u4.test         
  inflating: ml-100k/u5.base         
  inflating: ml-100k/u5.test         
  inflating: ml-100k/ua.base         
  inflating: ml-100k/ua.test         
  inflating: ml-100k/ub.base         
  inflating: ml-

In [7]:
#Types of genres
genre = pd.read_csv('./ml-100k/u.genre', sep="|", encoding='latin-1', header=None)
genre.drop(genre.columns[1], axis=1, inplace=True)
genre.columns = ['Genres']
genre_list = list(genre['Genres'])
genre_list

['unknown',
 'Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [8]:
#Types of occupations
occupation = pd.read_csv('./ml-100k/u.occupation', sep="|", encoding='latin-1', header=None)
occupation.columns = ['Occupations']
occupation_list = list(occupation['Occupations'])
occupation_list

['administrator',
 'artist',
 'doctor',
 'educator',
 'engineer',
 'entertainment',
 'executive',
 'healthcare',
 'homemaker',
 'lawyer',
 'librarian',
 'marketing',
 'none',
 'other',
 'programmer',
 'retired',
 'salesman',
 'scientist',
 'student',
 'technician',
 'writer']

In [9]:
#Load the Ratings data
data = pd.read_csv('./ml-100k/u.data', sep="\t", header=None)
data.columns = ['userId', 'movieId', 'rating', 'timestamp']
data.head()

,userId,movieId,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [10]:
#Load the Movies data
item = pd.read_csv('./ml-100k/u.item', sep="|", encoding='latin-1', header=None)
item.columns = ['movieId', 'title' ,'release','video release date', 'IMDb URL', 'unknown', 'Action', 
                'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 
                'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
item['release'] = pd.to_datetime(item['release'])
item = item[pd.notnull(item['release'])]
item['year'] = item['release'].dt.year.astype(int)
item.drop(columns=['release', 'video release date', 'IMDb URL'], inplace=True)
item.head()

,movieId,title,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year
0,1,Toy Story (1995),0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1995
1,2,GoldenEye (1995),0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1995
2,3,Four Rooms (1995),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1995
3,4,Get Shorty (1995),0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1995
4,5,Copycat (1995),0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1995


In [11]:
df_meta = item.melt(id_vars=['movieId', 'title'], var_name='genre')
df_meta = df_meta[df_meta.value == 1]
df_meta.drop(columns=['value'], inplace=True)
df_meta.head()

,movieId,title,genre
1371,1373,Good Morning (1971),unknown
1682,2,GoldenEye (1995),Action
1684,4,Get Shorty (1995),Action
1697,17,From Dusk Till Dawn (1996),Action
1701,21,Muppet Treasure Island (1996),Action


In [12]:
#Load the User data
user = pd.read_csv('./ml-100k/u.user', sep="|", encoding='latin-1', header=None)
user.columns = ['userId', 'age', 'gender', 'occupation', 'zip code']
user.head()

,userId,age,gender,occupation,zip code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [15]:
movies = pd.read_csv('./dataset/movies_sample.csv', names=['itemId', 'title', 'genre'], header=0)
ratings = pd.read_csv('./dataset/ratings_sample.csv', names=['userId', 'itemId', 'rating', 'timestamp'], header=0)
df = ratings[['userId', 'itemId', 'rating']]
df = df.merge(movies[['itemId', 'title']])
df

,userId,itemId,rating,title
0,11,7481,5.0,Enemy Mine (1985)
1,11,1046,4.5,Beautiful Thing (1996)
2,11,616,4.0,"Aristocats, The (1970)"
3,11,3535,2.0,American Psycho (2000)
4,11,5669,5.0,Bowling for Columbine (2002)
...,...,...,...,...
190616,138493,288,5.0,Natural Born Killers (1994)
190617,138493,1748,5.0,Dark City (1998)
190618,138493,616,4.0,"Aristocats, The (1970)"
190619,138493,1597,4.5,Conspiracy Theory (1997)


In [16]:
map_users = {user: idx for idx, user in enumerate(df.userId.unique())}
map_items = {item: idx for idx, item in enumerate(df.itemId.unique())}
df['userId'] = df['userId'].map(map_users)
df['itemId'] = df['itemId'].map(map_items)
map_title = {}

for _, row in df.iterrows():
    map_title[row.itemId] = row.title


### Divisão da base em treino e teste

In [17]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=.2, random_state=2)
train.to_csv('train.txt', index=False, header=False, sep='\t')
test.to_csv('test.txt', index=False, header=False, sep='\t')

## Abordagens intrínsecas ao modelo

***Exercício 01:*** Utilize o algoritmo UserKNN (pode ser do CaseRecommender) para gerar recomendações para um usuário aleatório da base. Forneça uma explicação para uma das recomendações geradas. Dica: sua explicação pode ser na forma de um gráfico, veja o slide 13 "explicações baseadas em usuários ou itens relaventes".

**Construindo uma matriz de similaridade**

Vamos considerar os 5 vizinhos de um dado usuário para montar a explicação da recomendação

In [21]:
df

,userId,itemId,rating,title
0,0,0,5.0,Enemy Mine (1985)
1,0,1,4.5,Beautiful Thing (1996)
2,0,2,4.0,"Aristocats, The (1970)"
3,0,3,2.0,American Psycho (2000)
4,0,4,5.0,Bowling for Columbine (2002)
...,...,...,...,...
190616,11089,22,5.0,Natural Born Killers (1994)
190617,11089,10,5.0,Dark City (1998)
190618,11089,2,4.0,"Aristocats, The (1970)"
190619,11089,21,4.5,Conspiracy Theory (1997)


In [32]:
from numpy.linalg import norm


def cossine_sim(a,b):
    numerator = np.dot(a,b)
    denominator = norm(a) *norm(b)
    if (denominator != 0):
        return numerator / denominator 
    else:
        return 0

In [29]:
def get_user_vector(userId, movieList, df):
    users_ratings = df[df["itemId"].isin(movieList)]
    return users_ratings[users_ratings["userId"] == userId].rating.tolist()

In [30]:
def rated_by_user(user, df):
    return set(df[df["userId"] == user].itemId.tolist())

In [25]:
def get_commonly_rated_movies(userId1, userId2):
    rated_1 = rated_by_user(userId1, train)
    rated_2 = rated_by_user(userId2, train)

    return list(rated_1 & rated_2)

In [38]:
df.sort_values(by="rating", ascending=False).iloc[:5]

,userId,itemId,rating,title
0,0,0,5.0,Enemy Mine (1985)
76725,4453,16,5.0,Rush Hour 2 (2001)
76846,4460,17,5.0,Chasing Amy (1997)
76848,4460,40,5.0,Manon of the Spring (Manon des sources) (1986)
76852,4460,26,5.0,Being There (1979)


In [42]:
from itertools import combinations

def get_top_5_most_similar_users(userId, df):
    user_ids = sorted(df.userId.unique())
    sim_matrix = []
    for j in user_ids:
            inter_items = get_commonly_rated_movies(userId, j)
            vec_1 = get_user_vector(userId, inter_items, df)
            vec_2 = get_user_vector(j, inter_items, df)
            sim = cossine_sim(vec_1,vec_2)
            sim_matrix.append([userId, j, sim])
    return pd.DataFrame(sim_matrix, columns=['user1', 'user2', 'similarity']).sort_values(by="similarity", ascending=False).iloc[:5]

In [43]:
sim_matrix = get_top_5_most_similar_users(4453, train)

In [44]:
sim_matrix

,user1,user2,similarity
4337,4453,4337,1.0
281,4453,281,1.0
9767,4453,9767,1.0
9761,4453,9761,1.0
5878,4453,5878,1.0


In [45]:
df_pred[df_pred["userId"] == 4453]

,userId,movieId,pred
15324,4453,21,3.53788


In [73]:
for idx, r in sim_matrix.iterrows():
    user = int(r["user2"])
    neighbor_ratings = df[df["userId"] == user]
    rating_for_item = neighbor_ratings[neighbor_ratings["itemId"] == 21]
    print(rating_for_item.rating.values)

[]
[]
[2.5]
[]
[3.5]


In [ ]:
sim_matrix.to_csv('sim_matrix.dat', index=False, sep='\t', header=False)

In [18]:
from caserec.recommenders.rating_prediction.userknn import UserKNN

UserKNN('train.txt', 'test.txt', 'rp_uknn.txt', as_similar_first=True).compute()

[Case Recommender: Rating Prediction > UserKNN Algorithm]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 4.312482 sec
prediction_time:: 82.280382 sec
Eval:: MAE: 0.702269 RMSE: 0.902177 


In [19]:
df_pred = pd.read_csv('rp_uknn.txt', sep='\t', names=['userId', 'movieId', 'pred'])

In [20]:
df_pred

,userId,movieId,pred
0,0,0,3.602971
1,0,5,3.610243
2,1,13,3.455673
3,1,14,3.622683
4,1,16,2.974314
...,...,...,...
38120,11088,21,3.462404
38121,11089,12,4.379341
38122,11089,21,3.322800
38123,11089,57,2.731238


Vamos explicar as recomendações geradas. É importante ressaltar que o método visto em sala pode falhar se os top5 vizinhos não avaliaram o filme recomendado.

In [74]:
def explain_rec(userId, movieId, train):
    top5 = get_top_5_most_similar_users(userId, train)
    print(f"Achamos que você vai gostar do filme {movieId}. Aqui esta como os 5 usuários mais similares a você avaliaram esse filme")
    for idx, r in top5.iterrows():
        user = int(r["user2"])
        neighbor_ratings = df[df["userId"] == user]
        rating_for_item = neighbor_ratings[neighbor_ratings["itemId"] == movieId]
        print(rating_for_item.rating.values)

In [76]:
explain_rec(4453, 21, train)

Achamos que você vai gostar do filme 21. Aqui esta como os 5 usuários mais similares a você avaliaram esse filme
[]
[]
[2.5]
[]
[3.5]


***Exercício 02:*** Seguindo a mesma ideia do exercício anterior, forneça uma explicação de uma recomendação para um usuário com base no algoritmo ItemKNN.

In [77]:
df

,userId,itemId,rating,title
0,0,0,5.0,Enemy Mine (1985)
1,0,1,4.5,Beautiful Thing (1996)
2,0,2,4.0,"Aristocats, The (1970)"
3,0,3,2.0,American Psycho (2000)
4,0,4,5.0,Bowling for Columbine (2002)
...,...,...,...,...
190616,11089,22,5.0,Natural Born Killers (1994)
190617,11089,10,5.0,Dark City (1998)
190618,11089,2,4.0,"Aristocats, The (1970)"
190619,11089,21,4.5,Conspiracy Theory (1997)


In [86]:
def get_item_vector(itemId, usersList, df):
    item_ratings = df[df["userId"].isin(usersList)]
    return item_ratings[item_ratings["itemId"] == itemId].rating.tolist()

def rated_the_item(itemId, df):
    return set(df[df["itemId"] == itemId].userId.tolist())

def get_users_who_rated_both(itemId1, itemId2):
    rated_1 = rated_the_item(itemId1, train)
    rated_2 = rated_the_item(itemId2, train)

    return list(rated_1 & rated_2)

In [100]:
from itertools import combinations

def get_top_5_most_similar_items(itemId, df, userId):
    """
        Dado um filme itemId, retorna os 5 filmes mais similares
        que o usuario userId avaliou.
    """
    item_ids = sorted(df.itemId.unique())
    sim_matrix = []
    rated_by_user = list(df[df["userId"] == userId]["itemId"])
    for j in item_ids:
        if j in rated_by_user:
                inter_users = get_users_who_rated_both(itemId, j)
                vec_1 = get_item_vector(itemId, inter_users, df)
                vec_2 = get_item_vector(j, inter_users, df)
                sim = cossine_sim(vec_1,vec_2)
                sim_matrix.append([itemId, j, sim])
        else:
            pass
    return pd.DataFrame(sim_matrix, columns=['item1', 'item2', 'similarity']).sort_values(by="similarity", ascending=False).iloc[:5]

In [82]:
from caserec.recommenders.rating_prediction.itemknn import ItemKNN

ItemKNN('train.txt', 'test.txt', 'rp_iknn.txt', as_similar_first=True).compute()

[Case Recommender: Rating Prediction > ItemKNN Algorithm]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 5.428387 sec
prediction_time:: 2.465187 sec
Eval:: MAE: 0.663504 RMSE: 0.872175 


In [83]:
df_pred_item = pd.read_csv('rp_iknn.txt', sep='\t', names=['userId', 'movieId', 'pred'])

In [87]:
get_top_5_most_similar_items(0, train)

,item1,item2,similarity
317,0,322,1.0
319,0,324,1.0
293,0,296,1.0
314,0,319,1.0
357,0,365,1.0


In [106]:
def explain_rec_item(userId, movieId, train):
    top5 = get_top_5_most_similar_items(movieId, train, userId)
    print(f"Achamos que você vai gostar do filme {movieId}. Aqui está como você avaliou os 5 filmes mais similares a este")
    for idx, r in top5.iterrows():
        item = int(r["item2"])
        user_rating = df[df["userId"] == userId]
        rating_for_item = user_rating[user_rating["itemId"] == movieId]
        print(rating_for_item.rating.values)

In [107]:
explain_rec_item(0, 0, train)

Achamos que você vai gostar do filme 0. Aqui está como você avaliou os 5 filmes mais similares a este
[5.]
[5.]
[5.]
[5.]
[5.]


***Exercício 03:*** Forneça uma explicação de uma recomendação para um usuário com base no algoritmo ItemAttributeKNN. Utilize o template: "Estou te recomendando X pois este filme possui os mesmos gêneros A, B e C que o filme Y que você já assistiu e gostou".
- X, Y : título de um filme
- A, B, C : gênero

In [111]:
item[item["movieId"] == 1]

,movieId,title,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year
0,1,Toy Story (1995),0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1995


In [148]:

def get_movie_genders(movieId, df):
    generos = []
    cols_to_ignore = ["movieId", "title"]
    for _, row in df[df["movieId"] == movieId].iterrows():
        generos = [col for col in teste.columns if row[col] == 1 and col not in cols_to_ignore]
    return generos

In [128]:
def get_top_k_rated_movies(userId, df, k=1):
    movies = df[df["userId"] == userId]
    return movies.sort_values(by="rating", ascending=False)[:k]

In [144]:
get_top_k_rated_movies(0, train)

9

In [145]:
def explain_by_template(movieId, userId):
    top_rated_movie = get_top_k_rated_movies(userId, train)["itemId"].values[0]
    genders = get_movie_genders(top_rated_movie, item)
    recomended_movie_genders = get_movie_genders(movieId, item)

    genders_inter = list(set(genders).intersection(set(recomended_movie_genders)))
    print(f"Estou te recomendando o filme {movieId} porque ele possui os generos {genders_inter}, assim como o filme {top_rated_movie}, que você ja viu e gostou")

In [133]:
item

,movieId,title,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year
0,1,Toy Story (1995),0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1995
1,2,GoldenEye (1995),0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1995
2,3,Four Rooms (1995),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1995
3,4,Get Shorty (1995),0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1995
4,5,Copycat (1995),0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1998
1678,1679,B. Monkey (1998),0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,1998
1679,1680,Sliding Doors (1998),0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1998
1680,1681,You So Crazy (1994),0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1994


In [154]:
df_pred_item

,userId,movieId,pred
0,0,0,4.150225
1,0,5,3.832249
2,1,13,3.917192
3,1,14,4.107512
4,1,16,3.418735
...,...,...,...
38120,11088,21,4.336038
38121,11089,12,5.000000
38122,11089,21,4.136737
38123,11089,57,3.597058


In [157]:
get_movie_genders(12, item)

['Crime', 'Thriller']

In [158]:
get_movie_genders(21, item)

['Action', 'Adventure', 'Comedy', 'Musical', 'Thriller']

In [156]:
explain_by_template(21, 11088)

Estou te recomendando o filme 21 porque ele possui os generos ['Thriller'], assim como o filme 12, que você ja viu e gostou


## Abordagem agnóstica ao modelo

***Exercício 04:*** Pense e implemente uma estratégia de explicação agnóstica ao modelo: utilize o BPRMF para calcular recomencações, e depois implemente outra estratégia para explicar uma das recomendações para um usuário.

In [12]:
# TODO